# Self-Learning AI - Cloud Training

This notebook runs on Google Colab or Kaggle to perform GPU-intensive training.

**Setup:**
1. Upload your learning data
2. Run training on free GPU
3. Download improved model
4. Load it back on your M3 Mac

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate peft bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import json

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## Load Model

In [ ]:
# Model configuration
MODEL_NAME = "meta-llama/Llama-3.2-3B"  # or your preferred model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

print(f"Model loaded: {MODEL_NAME}")
print(f"Parameters: {model.num_parameters():,}")

## Prepare for LoRA Training

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## Load Training Data

Upload your `training_data.json` file from your M3 Mac

In [ ]:
# For Google Colab - upload file
from google.colab import files
uploaded = files.upload()

# Load training data
with open('training_data.json', 'r') as f:
    training_data = json.load(f)

print(f"Loaded {len(training_data)} training examples")

In [ ]:
# Prepare dataset
def format_example(example):
    text = f"Question: {example['prompt']}\nAnswer: {example['completion']}"
    return tokenizer(text, truncation=True, max_length=512, padding="max_length")

# Create dataset
dataset = Dataset.from_list(training_data)
tokenized_dataset = dataset.map(format_example, remove_columns=dataset.column_names)

print(f"Dataset prepared with {len(tokenized_dataset)} examples")

## Train!

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_steps=100,
    logging_steps=10,
    save_total_limit=2,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Start training
print("Starting training...")
trainer.train()

## Save and Download Model

In [ ]:
# Save model
output_dir = "./trained_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

In [ ]:
# Create zip for download
!zip -r trained_model.zip trained_model/

# Download (for Colab)
from google.colab import files
files.download('trained_model.zip')

print("Download the model and extract it to your M3 Mac's data/models/ directory")

## Test the Model

In [ ]:
# Test generation
test_prompt = "Question: What is machine learning?\nAnswer:"
inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_length=200,
    temperature=0.7,
    do_sample=True
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)